In [20]:
import cv2 as cv
import numpy as np

img = cv.imread('train/train/clips-30049.png')
# Convertir la imagen a escala de grises
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Aplicar un umbral adaptativo a la imagen en escala de grises
thresh = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 11, 2)

# Aplicar la transformada de Hough para detectar las líneas horizontales
horizontal_lines = cv.HoughLinesP(thresh, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Crear una máscara para las líneas horizontales detectadas
mask_horizontal = np.ones_like(thresh, dtype=np.uint8)

for line in horizontal_lines:
    x1, y1, x2, y2 = line[0]
    if abs(y2 - y1) > abs(x2 - x1):
        cv.line(mask_horizontal, (x1, y1), (x2, y2), 0, 2)

# Aplicar la máscara de líneas horizontales a la imagen binaria
masked_thresh_horizontal = cv.bitwise_and(thresh, thresh, mask=mask_horizontal)

# Aplicar la transformada de Hough para detectar las líneas verticales
vertical_lines = cv.HoughLinesP(masked_thresh_horizontal, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Crear una máscara para las líneas verticales detectadas
mask_vertical = np.ones_like(thresh, dtype=np.uint8)

for line in vertical_lines:
    x1, y1, x2, y2 = line[0]
    if abs(x2 - x1) > abs(y2 - y1):
        cv.line(mask_vertical, (x1, y1), (x2, y2), 0, 2)

# Aplicar la máscara de líneas verticales a la imagen binaria
masked_thresh_vertical = cv.bitwise_and(masked_thresh_horizontal, masked_thresh_horizontal, mask=mask_vertical)

# Realizar una operación de apertura para eliminar el ruido
kernel = np.ones((5, 5), np.uint8)
opening = cv.morphologyEx(masked_thresh_vertical, cv.MORPH_OPEN, kernel, iterations=2)

# Encontrar los contornos de los objetos restantes en la imagen
contours, _ = cv.findContours(opening.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Crear una imagen en blanco para dibujar los contornos
result = np.zeros_like(img)

# Dibujar los contornos encontrados en la imagen en blanco
cv.drawContours(result, contours, -1, (0, 255, 0), 2)

# Mostrar la imagen original, la imagen binaria con las líneas horizontales y verticales eliminadas y los contornos encontrados
cv.imshow('Original', img)
cv.imshow('Thresholded Image', masked_thresh_vertical)
cv.imshow('Contours', result)

# Esperar a que se presione cualquier tecla y luego cerrar las ventanas
cv.waitKey(0)
cv.destroyAllWindows()

